In [88]:
%matplotlib inline
import matplotlib
import seaborn as sns
import pandas as pd
import sqlalchemy
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [2]:
from static_grader import grader

# SQL Miniproject


## Introduction


The city of New York does restaurant inspections and assigns a grade. Inspections data for the last 4 years are available on s3 as an SQLite database, which you can import in the next few cells. These were extracted from a set of csv files and an XLS file, as described in the <b>How we loaded the data</b> section


The raw data can be found [here](https://s3.amazonaws.com/dataincubator-course/coursedata/nyc_inspection_data.zip) and can be useful to look at. The file `RI_Webextract_BigApps_Latest.xls` contains a description of each of the data files and what the columns mean.

In [3]:
!aws s3 sync s3://dataincubator-course/coursedata/ . --exclude '*' --include 'nyc_inspection.db'

download: s3://dataincubator-course/coursedata/nyc_inspection.db to ./nyc_inspection.db


In [4]:
#This will load the pre-existing tables
%load_ext sql
%sql sqlite:///nyc_inspection.db

'Connected: @nyc_inspection.db'

To see what tables are in the database:

In [5]:
%%sql
SELECT * FROM sqlite_master WHERE "type"='table';

 * sqlite:///nyc_inspection.db
Done.


type,name,tbl_name,rootpage,sql
table,writer,writer,2,"CREATE TABLE writer (first_name, last_name, year)"
table,webextract,webextract,3,"CREATE TABLE webextract ( ""index"" BIGINT, ""CAMIS"" BIGINT, ""DBA"" TEXT, ""BORO"" BIGINT, ""BUILDING"" TEXT, ""STREET"" TEXT, ""ZIPCODE"" FLOAT, ""PHONE"" TEXT, ""CUISINECODE"" BIGINT, ""INSPDATE"" TEXT, ""ACTION"" TEXT, ""VIOLCODE"" TEXT, ""SCORE"" FLOAT, ""CURRENTGRADE"" TEXT, ""GRADEDATE"" TEXT, ""RECORDDATE"" TEXT)"
table,violations,violations,24393,"CREATE TABLE violations ( ""index"" BIGINT, ""STARTDATE"" TEXT, ""ENDDATE"" TEXT, ""CRITICALFLAG"" TEXT, ""VIOLATIONCODE"" TEXT, ""VIOLATIONDESC"" TEXT)"
table,cuisine,cuisine,24426,"CREATE TABLE cuisine ( ""index"" BIGINT, ""CUISINECODE"" BIGINT, ""CODEDESC"" TEXT)"
table,boroughs,boroughs,24428,"CREATE TABLE boroughs ( id INT, name TEXT)"


And to look at the format of an individual table (note that you may need to change types to get the answers in the right form):

In [6]:
%%sql
PRAGMA table_info(webextract)

 * sqlite:///nyc_inspection.db
Done.


cid,name,type,notnull,dflt_value,pk
0,index,BIGINT,0,None,0
1,CAMIS,BIGINT,0,None,0
2,DBA,TEXT,0,None,0
3,BORO,BIGINT,0,None,0
4,BUILDING,TEXT,0,None,0
5,STREET,TEXT,0,None,0
6,ZIPCODE,FLOAT,0,None,0
7,PHONE,TEXT,0,None,0
8,CUISINECODE,BIGINT,0,None,0
9,INSPDATE,TEXT,0,None,0


## SQLite3


The project should be written in SQL. Between SQLite and PostgreSQL we recommend SQLite for this project.  You can use the SQLite command prompt by running this command in bash
```bash
sqlite3 cmd "DROP TABLE IF EXISTS writer;\
CREATE TABLE IF NOT EXISTS writer (first_name, last_name, year);\
INSERT INTO writer VALUES ('William', 'Shakespeare', 1616);\
INSERT INTO writer VALUES ('Francis', 'Fitzgerald', 1896);\
\
SELECT * FROM writer;\
"
```
Alternatively, you can run bash commands in a Jupyter notebook by prepending the `!` in a code cell (notice that we conveniently get the output displayed

In [7]:
!sqlite3 cmd """\
DROP TABLE IF EXISTS writer;\
CREATE TABLE IF NOT EXISTS writer (first_name, last_name, year);\
INSERT INTO writer VALUES ('William', 'Shakespeare', 1616);\
INSERT INTO writer VALUES ('Francis', 'Fitzgerald', 1896);\
\
SELECT * FROM writer;\
"""

William|Shakespeare|1616
Francis|Fitzgerald|1896


Finally, we use the [`ipython-sql` extension](https://github.com/catherinedevlin/ipython-sql#ipython-sql) by first loading this extension and then running our code with the "magic" command in the first line
```python
%%sql sqlite://
```
Notice that the output table is formatted nicely as a nice HTML table.

This is our recommended technique.  However, the grader is expecting python objects and you may need to use list comprehensions to reformat this output

In [98]:
%%sql 
DROP TABLE IF EXISTS writer;
CREATE TABLE IF NOT EXISTS writer (first_name, last_name, year);
INSERT INTO writer VALUES ('William', 'Shakespeare', 1616);
INSERT INTO writer VALUES ('Francis', 'Fitzgerald', 1896);

SELECT * FROM writer;

 * sqlite:///nyc_inspection.db
Done.
Done.
1 rows affected.
1 rows affected.
Done.


first_name,last_name,year
William,Shakespeare,1616
Francis,Fitzgerald,1896


In [47]:
result = _
#This captures the output of the previous cell

In [87]:
type(result)

sql.run.ResultSet

In [96]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///nyc_inspection.db')
df = pd.read_sql_table('writer', engine)
df

,first_name,last_name,year
0,William,Shakespeare,1616
1,Francis,Fitzgerald,1896


## How we loaded the data


For future reference, here is how you can load data in to SQL (with examples).  If you have a csv file you created with something like

```
!printf "Name,Age\nAlice,3\nBob,10" > sample.csv.nogit
```


Then SQLite has a convenient [`.import` function](https://sqlite.org/cli.html#csv_import) which can create tables from `.csv` files.

```bash
sqlite> .import sample.csv.nogit sample
sqlite> SELECT * FROM sample;
```

The files may contain badly formatted text.  Unfortunately, this is all too common.  As a stop gap, remember that [`iconv`](https://linux.die.net/man/1/iconv) is a Unix utility that can convert files between different text encodings.

Alternatively, you can also read csv files using pandas and convert that into SQL via some SQL magic (this is what we actually did).

```
import pandas as pd
sample = pd.read_csv('sample.csv.nogit')
%sql DROP TABLE IF EXISTS sample
%sql PERSIST sample
%sql SELECT * FROM sample;
```

## Question 1: Null entries


Return the number of inspections (`CAMIS`, `INSPDATE` pairs) that do not have a score - i.e. where none of the rows with those (`CAMIS`, `INSPDATE`) values has a score. Remove the corresponding rows from the data set for the rest of the questions in the assignment.

**Question:** How else might we have handled this?

In [131]:
%%sql
DROP TABLE IF EXISTS null_results;
CREATE TABLE IF NOT EXISTS null_results AS
    SELECT CAMIS, INSPDATE
    FROM webextract
    WHERE (SCORE IS NULL);

 * sqlite:///nyc_inspection.db
Done.
Done.


[]

In [126]:
#from sqlalchemy import create_engine
#engine = create_engine('sqlite:///nyc_inspection.db')
null_results = pd.read_sql_table('null_results', engine)
null_results

,CAMIS,INSPDATE
0,30075445,2011-04-27 00:00:00
1,30075445,2011-11-12 00:00:00
2,30075445,2011-11-12 00:00:00
3,40356483,2013-08-03 00:00:00
4,30075445,2012-05-03 00:00:00
5,30075445,2013-06-01 00:00:00
6,30075445,2011-12-21 00:00:00
7,30075445,2012-12-31 00:00:00
8,40356151,2012-02-10 00:00:00
9,30191841,2010-09-23 00:00:00


In [128]:
len(null_results)

33524

In [129]:
def null_entries():
    return len(null_results)

grader.score('sql__null_entries', null_entries)

Your score:  0


In [136]:
%%sql
DELETE FROM webextract WHERE SCORE IS NULL;

 * sqlite:///nyc_inspection.db
33524 rows affected.


[]

## Question 2: Score by zip code


Return a list of tuples of the form:

    (zip code, mean score, number of restaurants)

for each of the 87 zip codes in the city with over 100 restaurants. Use the score from the latest inspection date for each restaurant. Sort the list in ascending order by mean score.

**Note:** There is an interesting discussion here about what the mean score *means* in this data set. Think about what we're actually calculating - does it represent what we're trying to understand about these zip codes?

What if we use the average of a restaurant's inspections instead of the latest?

**Checkpoints:**
- Total unique restaurants: 24,361;
- Total restaurants in valid zip codes: 19,172


In [372]:
%%sql
DROP TABLE IF EXISTS zip_scores;
CREATE TABLE IF NOT EXISTS zip_scores AS
    SELECT ZIPCODE, SCORE, MAX(GRADEDATE) AS recent_date
    FROM webextract
    GROUP BY CAMIS;

 * sqlite:///nyc_inspection.db
Done.
Done.


[]

In [373]:
%%sql
DROP TABLE IF EXISTS zipcode;
CREATE TABLE IF NOT EXISTS zipcode AS
    SELECT ZIPCODE, AVG(SCORE), COUNT(*)
        FROM zip_scores
        GROUP BY ZIPCODE
        HAVING COUNT(*) > 100

 * sqlite:///nyc_inspection.db
Done.
Done.


[]

In [374]:
zipcode = pd.read_sql_table('zipcode', engine)
zipcode['ZIPCODE'] = zipcode['ZIPCODE'].astype(int)
#zipcode['ZIPCODE'] = zipcode['ZIPCODE'].astype(str)
zipcode

,ZIPCODE,AVG(SCORE),COUNT(*)
0,10001,8.785859,495
1,10002,11.517621,454
2,10003,11.504559,658
3,10004,10.826772,127
4,10007,11.068702,131
5,10009,11.260726,303
6,10010,11.016736,239
7,10011,11.034014,441
8,10012,11.162025,395
9,10013,11.556034,464


In [365]:
zipcode.sort_values(by=['AVG(SCORE)'])

,ZIPCODE,AVG(SCORE),COUNT(*)


In [366]:
def score_by_zipcode():
    return zipcode.sort_values(by=['AVG(SCORE)'])
#[("11201", 9.81739130434783, 345)] * 87

grader.score('sql__score_by_zipcode', score_by_zipcode)

[] is too short

Failed validating 'minItems' in schema:
    {'items': {'items': [{'type': 'string'},
                         {'type': 'number'},
                         {'minimum': 0, 'type': 'integer'}],
               'maxItems': 3,
               'minItems': 3,
               'type': 'array'},
     'maxItems': 87,
     'minItems': 87,
     'type': 'array'}

On instance:
    []


## Question 3: Mapping scores


The above are not terribly enlightening.  Use [CartoDB](http://cartodb.com/) to produce a map of average scores by zip code.  You can sign up for a free trial.

You will have to use their wizard to plot the data by [zip code](https://carto.com/learn/guides/analysis/georeference). You will need to specify "USA" in the country field.  Then use the "share" button to return a link of the form [https://x.cartodb.com/](https://x.cartodb.com/).

**For fun:** How do JFK, Brighton Beach, Liberty Island (home of the Statue of Liberty), Financial District, Chinatown, and Coney Island fare?

**For more fun:** Plot restaurants as pins on the map, allowing the user to filter by "low", "middling", or "high"-scoring restaurants. You can use a CASE WHEN statement to create the different groups based on score thresholds.

In [ ]:
def score_by_map():
    # must be url of the form https://x.cartodb.com/...
    return "https://cartodb.com"

grader.score('sql__score_by_map', score_by_map)

## Question 4: Score by borough

Return a list of tuples of the form:

    (borough, mean score, number of restaurants)

for each of the city's five boroughs. Use the latest score for each restaurant. Sort the list in ascending order by grade.

**Hint:** You will have to perform a join with the `boroughs` table. The borough names should be reported in ALL CAPS.

**Checkpoint:**
- Total restaurants in valid boroughs: 24,350

In [156]:
%%sql
DROP TABLE IF EXISTS boro_scores;
CREATE TABLE IF NOT EXISTS boro_scores AS
    SELECT BORO, SCORE, MAX(GRADEDATE) AS recent_date
    FROM webextract
    GROUP BY CAMIS;

 * sqlite:///nyc_inspection.db
Done.
Done.


[]

In [170]:
%%sql
SELECT boroughs.name, boro_scores.SCORE
FROM boro_scores
LEFT JOIN boroughs ON boro_scores.BORO=boroughs.id;

 * sqlite:///nyc_inspection.db
Done.


name,SCORE
THE BRONX,2.0
BROOKLYN,23.0
MANHATTAN,11.0
BROOKLYN,5.0
QUEENS,13.0
QUEENS,10.0
STATEN ISLAND,12.0
BROOKLYN,10.0
BROOKLYN,12.0
BROOKLYN,12.0


In [171]:
%%sql
DROP TABLE IF EXISTS boros;
CREATE TABLE IF NOT EXISTS boros AS
    SELECT boroughs.name, boro_scores.SCORE
    FROM boro_scores
    LEFT JOIN boroughs ON boro_scores.BORO=boroughs.id;

 * sqlite:///nyc_inspection.db
Done.
Done.


[]

In [172]:
%%sql
DROP TABLE IF EXISTS boro_avg;
CREATE TABLE IF NOT EXISTS boro_avg AS
    SELECT name, AVG(SCORE), COUNT(*)
        FROM boros
        GROUP BY name

 * sqlite:///nyc_inspection.db
Done.
Done.


[]

In [189]:
boro = pd.read_sql_table('boro_avg', engine)
boro = boro.drop(boro.index[0])
boro = boro.sort_values(by=['AVG(SCORE)'])
boro

,name,AVG(SCORE),COUNT(*)
5,THE BRONX,10.231039,2281
2,MANHATTAN,10.870286,9891
1,BROOKLYN,10.959651,5849
3,QUEENS,11.274506,5417
4,STATEN ISLAND,11.464912,912


In [190]:
def score_by_borough():
    return boro
#[("MANHATTAN", 10.7269875502402, 10201)] * 5

grader.score('sql__score_by_borough', score_by_borough)

Your score:  0.7


## Question 5: Violations by cuisine


We want to look at violations themselves now.  We'll need to think more carefully about what we're measuring, since most restaurants have many inspections with possibly multiple violations per inspection, or long stretches of inspections with no violations.

There are many ways to deal with this normalization issue, but we'll go with a fairly straightforward one: dividing the number of violations by the length of time (in years) the restaurant has been open.  As a proxy for the length, we'll look at the difference between the oldest and newest inspection date, treating anything less than 30 days as 30 days (to account for those that were only inspected once, we'll assume everything was open for at least a month).

Since there are so many restaurants, we'll group them by cuisine and do a weighted average by computing 

    (total violations for a cuisine) / (total restaurant-years for that cuisine)

Return a list of 75 tuples of the form

    (cuisine name, reports per restaurant-year)
    
for cuisines with at least 100 violations total, ordered by increasing number of reports per restaurant-year
    
**Note:** This isn't the only way to normalize things.  How would other ways affect the computation?  If you similarly wanted to compute an average score by cuisine, how might you go about doing that?
    
**Checkpoint:**
- Total entries from valid cuisines: 522,410

In [198]:
%%sql
DROP TABLE IF EXISTS tot_cuisines;
CREATE TABLE IF NOT EXISTS tot_cuisines AS
    SELECT CAMIS, CUISINECODE, GRADEDATE
    FROM webextract

 * sqlite:///nyc_inspection.db
Done.
Done.


[]

In [199]:
%%sql
DROP TABLE IF EXISTS tot_cuisines2;
CREATE TABLE IF NOT EXISTS tot_cuisines2 AS
    SELECT tot_cuisines.CAMIS, tot_cuisines.GRADEDATE, cuisine.CODEDESC
    FROM tot_cuisines
    LEFT JOIN cuisine ON tot_cuisines.CUISINECODE=cuisine.CUISINECODE;

 * sqlite:///nyc_inspection.db
Done.
Done.


[]

In [224]:
%%sql
DROP TABLE IF EXISTS tot_cuisines3;
CREATE TABLE IF NOT EXISTS tot_cuisines3 AS
    SELECT CAMIS, CODEDESC, COUNT(*), JULIANDAY(MAX(GRADEDATE)) - JULIANDAY(MIN(GRADEDATE)) AS DIFFERENCE FROM tot_cuisines2
    GROUP BY CAMIS;

 * sqlite:///nyc_inspection.db
Done.
Done.


[]

In [229]:
%%sql
DROP TABLE IF EXISTS camis_avg;
CREATE TABLE IF NOT EXISTS camis_avg AS
    SELECT CODEDESC, COUNT(*) - 1 AS AVG FROM tot_cuisines3
    GROUP BY CAMIS;

 * sqlite:///nyc_inspection.db
Done.
Done.


[]

In [225]:
%%sql
SELECT * FROM tot_cuisines3
LIMIT 3;

 * sqlite:///nyc_inspection.db
Done.


CAMIS,CODEDESC,COUNT(*),DIFFERENCE
30075445,Bakery,22,1089.0
30112340,Hamburgers,19,930.0
30191841,American,31,1067.0


In [247]:
camis_avg = pd.read_sql_table('tot_cuisines3', engine)
camis_avg

,CAMIS,CODEDESC,COUNT(*),DIFFERENCE
0,30075445,Bakery,22,1089.0
1,30112340,Hamburgers,19,930.0
2,30191841,American,31,1067.0
3,40356018,American,14,972.0
4,40356068,Jewish/Kosher,27,568.0
5,40356151,American,22,986.0
6,40356442,Jewish/Kosher,11,847.0
7,40356483,Delicatessen,34,958.0
8,40356649,American,21,1066.0
9,40356731,"Ice Cream, Gelato, Yogurt, Ices",11,872.0


In [251]:
cuisine_sum = camis_avg.groupby('CODEDESC').sum()
cuisine_sum

,CAMIS,COUNT(*),DIFFERENCE
CODEDESC,,,
Afghan,620663014,276,7855.0
African,3252533652,1788,46983.0
American,262331091987,119202,4421776.0
Armenian,2231014294,574,19525.0
Asian,14148173376,7339,185484.0
Australian,664580810,233,7270.0
Bagels/Pretzels,6928286462,3533,123761.0
Bakery,28317041274,15513,514609.0
Bangladeshi,1254128952,779,18697.0


In [255]:
cuisine_sum['AVG'] = cuisine_avg['COUNT(*)']/(cuisine_avg['DIFFERENCE']/365)
cuisine_sum

,CAMIS,COUNT(*),DIFFERENCE,AVG
CODEDESC,,,,
Afghan,620663014,276,7855.0,12.824952
African,3252533652,1788,46983.0,13.705349
American,262331091987,119202,4421776.0,9.646183
Armenian,2231014294,574,19525.0,10.511359
Asian,14148173376,7339,185484.0,14.216914
Australian,664580810,233,7270.0,11.698074
Bagels/Pretzels,6928286462,3533,123761.0,10.355715
Bakery,28317041274,15513,514609.0,10.953590
Bangladeshi,1254128952,779,18697.0,15.207520


In [259]:
cuisine_avg = cuisine_sum.drop(columns=['CAMIS', 'DIFFERENCE'])
cuisine_avg

,COUNT(*),AVG
CODEDESC,,
Afghan,276,12.824952
African,1788,13.705349
American,119202,9.646183
Armenian,574,10.511359
Asian,7339,14.216914
Australian,233,11.698074
Bagels/Pretzels,3533,10.355715
Bakery,15513,10.953590
Bangladeshi,779,15.207520


In [287]:
cuisavg_100 = cuisine_avg.loc[cuisine_avg['COUNT(*)'] > 100]
cuisavg_100

,COUNT(*),AVG
CODEDESC,,
Afghan,276,12.824952
African,1788,13.705349
American,119202,9.646183
Armenian,574,10.511359
Asian,7339,14.216914
Australian,233,11.698074
Bagels/Pretzels,3533,10.355715
Bakery,15513,10.953590
Bangladeshi,779,15.207520


In [288]:
cuisavg_100 = cuisavg_100.drop(columns=['COUNT(*)'])

In [289]:
cuisavg_100 = cuisavg_100.sort_values(by=['AVG'])
cuisavg_100

,AVG
CODEDESC,
Hotdogs/Pretzels,4.355875
Donuts,5.955119
Soups & Sandwiches,6.051493
"Ice Cream, Gelato, Yogurt, Ices",6.186705
Sandwiches,6.622560
Café/Coffee/Tea,6.711127
Not Listed/Not Applicable,6.823677
Hotdogs,6.945130
"Bottled beverages, including water, sodas, juices, etc.",7.072222


In [290]:
cuisavg_100['AVG'] = cuisavg_100['AVG'].astype(str)
cuisavg_100

,AVG
CODEDESC,
Hotdogs/Pretzels,4.355875152998776
Donuts,5.95511894311671
Soups & Sandwiches,6.051492733705664
"Ice Cream, Gelato, Yogurt, Ices",6.186704891799307
Sandwiches,6.622559679309581
Café/Coffee/Tea,6.711126821420121
Not Listed/Not Applicable,6.82367709069637
Hotdogs,6.9451302571088025
"Bottled beverages, including water, sodas, juices, etc.",7.072222222222222


In [291]:
def score_by_cuisine():
    return cuisavg_100
#[("French", 20.3550686378036)] * 75

grader.score('sql__score_by_cuisine', score_by_cuisine)

['4.355875152998776'] is too short

Failed validating 'minItems' in schema['items']:
    {'items': [{'type': 'string'}, {'type': 'number'}],
     'maxItems': 2,
     'minItems': 2,
     'type': 'array'}

On instance[0]:
    ['4.355875152998776']


## Question 6: Specific violations by cuisine

Which cuisines tend to have a disproportionate number of what which violations? Answering this question isn't easy because you have to think carefully about normalizations.

1. More popular cuisine categories will tend to have more violations just because they represent more restaurants.
2. Similarly, some violations are more common.  For example, knowing that "Equipment not easily movable or sealed to floor" is a common violation for Chinese restaurants is not particularly helpful when it is a common violation for all restaurants.

The right quantity is to look at is the conditional probability of a specific type of violation given a specific cuisine type and divide it by the unconditional probability of the violation for the entire population. Taking this ratio gives the right answer.  Return the 20 highest ratios of the form:

    ((cuisine, violation), ratio, count)

**Hint:**
1. You might want to check out this [Stack Overflow post](http://stackoverflow.com/questions/972877/calculate-frequency-using-sql).
2. The definition of a violation changes with time.  For example, 10A can mean two different things "Toilet facility not maintained ..." or "Vermin or other live animal present ..." when things were prior to 2003. To deal with this, you should limit your analysis to violation codes with end date after Jan 1, 2014. (This end date refers to the validity time ranges in `Violation.txt`).
3. The ratios don't mean much when the number of violations of a given type and for a specific category are not large (why not?).  Be sure to filter these out.  We chose 100 as our cutoff.

**Checkpoint:**
- Top 20 ratios mean: 2.360652529900757

In [315]:
%%sql
DROP TABLE IF EXISTS viol_cuisines;
CREATE TABLE IF NOT EXISTS viol_cuisines AS
    SELECT CAMIS, CUISINECODE, VIOLCODE, GRADEDATE
    FROM webextract

 * sqlite:///nyc_inspection.db
Done.
Done.


[]

In [316]:
%%sql
SELECT * FROM viol_cuisines
LIMIT 5;

 * sqlite:///nyc_inspection.db
Done.


CAMIS,CUISINECODE,VIOLCODE,GRADEDATE
30075445,8,10F,2014-03-03 00:00:00
30112340,39,06A,2014-07-01 00:00:00
30191841,3,10B,2013-07-22 00:00:00
40356483,27,08C,2014-05-29 00:00:00
30191841,3,02G,2013-07-22 00:00:00


In [333]:
%%sql
DROP TABLE IF EXISTS violcode;
CREATE TABLE IF NOT EXISTS violcode AS
    SELECT *
    FROM violations
    GROUP BY VIOLATIONCODE
    HAVING ENDDATE > '2014-01-01'

 * sqlite:///nyc_inspection.db
Done.
Done.


[]

In [334]:
%%sql
SELECT * FROM violcode
LIMIT 5;

 * sqlite:///nyc_inspection.db
Done.


index,STARTDATE,ENDDATE,CRITICALFLAG,VIOLATIONCODE,VIOLATIONDESC
39,2010-07-26 00:00:00,2099-12-31 00:00:00,Y,02A,Food not cooked to required minimum temperature.
46,2010-07-26 00:00:00,2099-12-31 00:00:00,Y,02B,Hot food item not held at or above 140º F.
53,2010-07-26 00:00:00,2099-12-31 00:00:00,Y,02C,Hot food item that has been cooked and refrigerated is being held for service without first being reheated to 1 65º F or above within 2 hours.
60,2010-07-26 00:00:00,2099-12-31 00:00:00,Y,02D,"Precooked potentially hazardous food from commercial food processing establishment that is supposed to be heated, but is not heated to 140º F within 2 hours."
67,2010-07-26 00:00:00,2099-12-31 00:00:00,Y,02E,"Whole frozen poultry or poultry breasts, other than a single portion, is being cooked frozen or partially thawed."


In [338]:
%%sql
DROP TABLE IF EXISTS violcode2;
CREATE TABLE IF NOT EXISTS violcode2 AS
    SELECT * FROM violcode

 * sqlite:///nyc_inspection.db
Done.
Done.


[]

In [340]:
%%sql
SELECT * FROM violcode2
LIMIT 5;

 * sqlite:///nyc_inspection.db
Done.


index,STARTDATE,ENDDATE,CRITICALFLAG,VIOLATIONCODE,VIOLATIONDESC
39,2010-07-26 00:00:00,2099-12-31 00:00:00,Y,02A,Food not cooked to required minimum temperature.
46,2010-07-26 00:00:00,2099-12-31 00:00:00,Y,02B,Hot food item not held at or above 140º F.
53,2010-07-26 00:00:00,2099-12-31 00:00:00,Y,02C,Hot food item that has been cooked and refrigerated is being held for service without first being reheated to 1 65º F or above within 2 hours.
60,2010-07-26 00:00:00,2099-12-31 00:00:00,Y,02D,"Precooked potentially hazardous food from commercial food processing establishment that is supposed to be heated, but is not heated to 140º F within 2 hours."
67,2010-07-26 00:00:00,2099-12-31 00:00:00,Y,02E,"Whole frozen poultry or poultry breasts, other than a single portion, is being cooked frozen or partially thawed."


In [330]:
%%sql
DROP TABLE IF EXISTS viol_cuisines2;
CREATE TABLE IF NOT EXISTS viol_cuisines2 AS
    SELECT viol_cuisines.CAMIS, cuisine.CODEDESC, viol_cuisines.VIOLCODE
    FROM viol_cuisines
    LEFT JOIN cuisine ON viol_cuisines.CUISINECODE=cuisine.CUISINECODE;

 * sqlite:///nyc_inspection.db
Done.
Done.


[]

In [329]:
%%sql
SELECT * FROM viol_cuisines2
LIMIT 5;

 * sqlite:///nyc_inspection.db
Done.


CAMIS,CODEDESC,VIOLCODE
30075445,Bakery,10F
30112340,Hamburgers,06A
30191841,American,10B
40356483,Delicatessen,08C
30191841,American,02G


In [343]:
%%sql
DROP TABLE IF EXISTS viol_cuisines3;
CREATE TABLE IF NOT EXISTS viol_cuisines3 AS
    SELECT viol_cuisines2.CAMIS, viol_cuisines2.CODEDESC, viol_cuisines2.VIOLCODE, violcode2.VIOLATIONDESC
    FROM viol_cuisines2
    LEFT JOIN violcode2 ON viol_cuisines2.VIOLCODE=violcode2.VIOLATIONCODE;

 * sqlite:///nyc_inspection.db
Done.
Done.


[]

In [344]:
%%sql
SELECT * FROM viol_cuisines3
LIMIT 5;

 * sqlite:///nyc_inspection.db
Done.


CAMIS,CODEDESC,VIOLCODE,VIOLATIONDESC
30075445,Bakery,10F,"Non-food contact surface improperly constructed. Unacceptable material used. Non-food contact surface or equipment improperly maintained and/or not properly sealed, raised, spaced or movable to allow accessibility for cleaning on all sides, above and underneath the unit."
30112340,Hamburgers,06A,Personal cleanliness inadequate. Outer garment soiled with possible contaminant. Effective hair restraint not worn in an area where food is prepared.
30191841,American,10B,Plumbing not properly installed or maintained; anti-siphonage or backflow prevention device not provided where required; equipment or floor not properly drained; sewage disposal system in disrepair or not functioning properly.
40356483,Delicatessen,08C,Pesticide use not in accordance with label or applicable laws. Prohibited chemical used/stored. Open bait station used.
30191841,American,02G,Cold food item held above 41º F (smoked fish and reduced oxygen packaged foods above 38 ºF) except during necessary preparation.


In [395]:
%%sql
DROP TABLE IF EXISTS viol_cuisines4;
CREATE TABLE IF NOT EXISTS viol_cuisines4 AS
    SELECT CODEDESC, VIOLCODE, VIOLATIONDESC, COUNT(*)
        FROM viol_cuisines3
        GROUP BY CODEDESC, VIOLCODE, VIOLATIONDESC
        HAVING COUNT(VIOLCODE) > 100;

 * sqlite:///nyc_inspection.db
Done.
(sqlite3.OperationalError) misuse of aggregate function COUNT() [SQL: 'CREATE TABLE IF NOT EXISTS viol_cuisines4 AS\n    SELECT CODEDESC, VIOLCODE, VIOLATIONDESC\n        FROM viol_cuisines3\n    WHERE COUNT(VIOLCODE) > 100;'] (Background on this error at: http://sqlalche.me/e/e3q8)


In [414]:
%%sql
DROP TABLE IF EXISTS viol_freq;
CREATE TABLE IF NOT EXISTS viol_freq AS
    SELECT  A.CODEDESC, A.VIOLCODE, A.COUNT1 * 1.0 / B.COUNT2 AS FREQ
    FROM    (
            SELECT CODEDESC, VIOLCODE, COUNT(*) As COUNT1
            FROM viol_cuisines3
            GROUP BY CODEDESC, VIOLCODE
            ) AS A
            INNER JOIN (
                SELECT CODEDESC, COUNT(*) As COUNT2
                FROM viol_cuisines3
                GROUP BY CODEDESC
                ) As B
                On A.CODEDESC = B.CODEDESC

 * sqlite:///nyc_inspection.db
Done.
Done.


[]

In [415]:
%%sql
SELECT * FROM viol_freq
LIMIT 5;

 * sqlite:///nyc_inspection.db
Done.


CODEDESC,VIOLCODE,FREQ
Afghan,None,0.0036231884057971015
Afghan,02B,0.014492753623188406
Afghan,02G,0.043478260869565216
Afghan,02H,0.0036231884057971015
Afghan,04A,0.021739130434782608


In [409]:
%%sql
DROP TABLE IF EXISTS viol_freq2;
CREATE TABLE IF NOT EXISTS viol_freq2 AS
    SELECT viol_freq.CODEDESC, violcode2.VIOLATIONDESC, viol_freq.FREQ
    FROM viol_freq
    LEFT JOIN violcode2 ON viol_freq.VIOLCODE=violcode2.VIOLATIONCODE;

 * sqlite:///nyc_inspection.db
Done.
Done.


[]

In [410]:
%%sql
SELECT * FROM viol_freq2
LIMIT 5;

 * sqlite:///nyc_inspection.db
Done.


CODEDESC,VIOLATIONDESC,FREQ
Afghan,None,0.0036231884057971015
Afghan,Hot food item not held at or above 140º F.,0.014492753623188406
Afghan,Cold food item held above 41º F (smoked fish and reduced oxygen packaged foods above 38 ºF) except during necessary preparation.,0.043478260869565216
Afghan,"Food not cooled by an approved method whereby the internal product temperature is reduced from 140º F to 70º F or less within 2 hours, and from 70º F to 41º F or less within 4 additional hours.",0.0036231884057971015
Afghan,Food Protection Certificate not held by supervisor of food operations.,0.021739130434782608


In [ ]:
def violation_by_cuisine():
    return [(("Café/Coffee/Tea",
              "Toilet facility not maintained and provided with toilet paper; "
              "waste receptacle and self-closing door."),
             1.87684775827172, 315)] * 20

grader.score('sql__violation_by_cuisine', violation_by_cuisine)

*Copyright &copy; 2016 The Data Incubator.  All rights reserved.*